In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import holidays

In [ ]:
STREET = 'washington'

df = pd.read_csv(f'data/{STREET}_merged.csv')

df['time'] = pd.to_datetime(df['time'])

print("Number of rows: ", len(df))
print("Columns: ", df.columns)

print("Start date: ", df['time'].min())
print("End date: ", df['time'].max())

In [ ]:
df.head(10)

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
# Handling missing values (simplest way)

df['throughput'] = df['throughput'].ffill()
df['throughput'] = df['throughput'].bfill()
df['occupancy'] = df['occupancy'].ffill()
df['occupancy'] = df['occupancy'].bfill()

# Remove unnecessary columns
df = df.drop(columns=['id_arc', 'id_upstream_node', 'upstream_node', 'id_downstream_node', 'downstream_node'])

df.isnull().sum()

In [ ]:
# Ensure 'time' is datetime
df['time'] = pd.to_datetime(df['time'])

# Extract features
df['hour'] = df['time'].dt.hour
df['day_of_week'] = df['time'].dt.dayofweek  # Monday=0, Sunday=6
df['month'] = df['time'].dt.month
df['day'] = df['time'].dt.day
df['week_of_year'] = df['time'].dt.isocalendar().week
df['year'] = df['time'].dt.year

# Define French holidays
fr_holidays = holidays.France()

def is_holiday(date):
    return 1 if date in fr_holidays else 0

df['is_holiday'] = df['time'].dt.date.apply(is_holiday)
df['is_weekend'] = df['day_of_week'].apply(lambda x: 1 if x >=5 else 0)

In [ ]:
# List of feature columns
feature_cols = ['hour', 'day_of_week', 'month', 'day', 'week_of_year', 'year', 'is_weekend', ] 

# Define target variables
targets = ['throughput', 'occupancy']

cutoff_date = df["time"].max() - pd.DateOffset(months=1, days=8)
max_date  = df["time"].max() - pd.DateOffset(months=1, days=3)

# Training set: Data up to cutoff_date
train_df = df[df['time'] <= cutoff_date]

# Testing set: Data after cutoff_date
test_df = df[(df['time'] > cutoff_date) & (df['time'] <= max_date)]

print(f"Training data shape: {train_df.shape}")
print(f"Testing data shape: {test_df.shape}")

# Print out the max date in the training set
print(f"Min date in training set: {train_df['time'].min()}")
print(f"Max date in training set: {train_df['time'].max()}")
print(f"Min date in testing set: {test_df['time'].min()}")
print(f"Max date in testing set: {test_df['time'].max()}")

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Features and target for throughput
X_train_throughput = train_df[feature_cols]
y_train_throughput = train_df['throughput']

X_test_throughput = test_df[feature_cols]
y_test_throughput = test_df['throughput']

# Handle any remaining missing values, if any
X_train_throughput = X_train_throughput.fillna(0)
X_test_throughput = X_test_throughput.fillna(0)

# Initialize the model
rf_throughput = XGBRegressor(n_estimators=100, random_state=42, n_jobs=-2)

# Train the model
rf_throughput.fit(X_train_throughput, y_train_throughput)

# Predict
y_pred_throughput = rf_throughput.predict(X_test_throughput)

# Evaluate
mae_throughput = mean_absolute_error(y_test_throughput, y_pred_throughput)
rmse_throughput = np.sqrt(mean_squared_error(y_test_throughput, y_pred_throughput))
print(f'Throughput - MAE: {mae_throughput}, RMSE: {rmse_throughput}')

In [ ]:
# Features and target for occupancy
X_train_occupancy = train_df[feature_cols]
y_train_occupancy = train_df['occupancy']

X_test_occupancy = test_df[feature_cols]
y_test_occupancy = test_df['occupancy']

# Handle any remaining missing values, if any
X_train_occupancy = X_train_occupancy.fillna(0)
X_test_occupancy = X_test_occupancy.fillna(0)

# Initialize the model
rf_occupancy = XGBRegressor(n_estimators=100, random_state=42, n_jobs=-2)

# Train the model
rf_occupancy.fit(X_train_occupancy, y_train_occupancy)

# Predict
y_pred_occupancy = rf_occupancy.predict(X_test_occupancy)

# Evaluate
mae_occupancy = mean_absolute_error(y_test_occupancy, y_pred_occupancy)
rmse_occupancy = np.sqrt(mean_squared_error(y_test_occupancy, y_pred_occupancy))
print(f'Occupancy - MAE: {mae_occupancy}, RMSE: {rmse_occupancy}')

In [ ]:
import matplotlib.pyplot as plt

# Plot for Throughput
plt.figure(figsize=(15,5))
plt.plot(test_df['time'], y_test_throughput, label='Actual Throughput')
plt.plot(test_df['time'], y_pred_throughput, label='Predicted Throughput')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Throughput')
plt.title('Actual vs Predicted Throughput')
plt.show()

# Plot for Occupancy
plt.figure(figsize=(15,5))
plt.plot(test_df['time'], y_test_occupancy, label='Actual Occupancy')
plt.plot(test_df['time'], y_pred_occupancy, label='Predicted Occupancy')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Occupancy')
plt.title('Actual vs Predicted Occupancy')
plt.show()

In [ ]:
from prophet import Prophet

df_throughput = df[['time', 'throughput']].rename(columns={'time': 'ds', 'throughput': 'y'})
df_occupancy = df[['time', 'occupancy']].rename(columns={'time': 'ds', 'occupancy': 'y'})

In [ ]:
def is_holiday(date):
    return 1 if date in fr_holidays else 0

def is_weekend(day_of_week):
    return 1 if day_of_week >= 5 else 0

# Add 'is_holiday' and 'is_weekend' to both dataframes
for df_forecast in [df_throughput, df_occupancy]:
    df_forecast['is_holiday'] = df_forecast['ds'].dt.date.apply(is_holiday)
    df_forecast['day_of_week'] = df_forecast['ds'].dt.dayofweek
    df_forecast['is_weekend'] = df_forecast['day_of_week'].apply(is_weekend)

In [ ]:
# From 1st October 2024 to 5th October 2024
cutoff_date = pd.to_datetime('2024-10-01')
max_date  = pd.to_datetime('2024-10-06')

train_throughput = df_throughput[df_throughput['ds'] <= cutoff_date]
test_throughput = df_throughput[(df_throughput['ds'] > cutoff_date) & (df_throughput['ds'] <= max_date)]

train_occupancy = df_occupancy[df_occupancy['ds'] <= cutoff_date]
test_occupancy = df_occupancy[(df_occupancy['ds'] > cutoff_date) & (df_occupancy['ds'] <= max_date)]

print(f"Throughput Training Data: {train_throughput.shape}")
print(f"Throughput Testing Data: {test_throughput.shape}")

print(f"Occupancy Training Data: {train_occupancy.shape}")
print(f"Occupancy Testing Data: {test_occupancy.shape}")

In [ ]:
FIT_PROPHET = False

In [ ]:
if FIT_PROPHET:
    # Initialize Prophet model
    model_throughput = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=True,
        seasonality_mode='additive',
        holidays=None  # We'll add regressors manually
    )

    # Add external regressors
    model_throughput.add_regressor('is_weekend')
    model_throughput.add_regressor('is_holiday')

    # Fit the model
    model_throughput.fit(train_throughput)

In [ ]:
if FIT_PROPHET:
    # Initialize Prophet model
    model_occupancy = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=True,
        seasonality_mode='additive',
        holidays=None  # We'll add regressors manually
    )

    # Add external regressors
    model_occupancy.add_regressor('is_weekend')
    model_occupancy.add_regressor('is_holiday')

    # Fit the model
    model_occupancy.fit(train_occupancy)

In [ ]:
# Define the forecast period
future_start = cutoff_date + pd.Timedelta(hours=1)  # Start right after the training period
future_end = max_date

future_dates = pd.date_range(start=future_start, end=future_end, freq='H')

future_throughput = pd.DataFrame({'ds': future_dates})
future_occupancy = pd.DataFrame({'ds': future_dates})

# Function to add external regressors to a future DataFrame
def add_external_regressors(df_future):
    # is_weekend
    df_future['day_of_week'] = df_future['ds'].dt.dayofweek
    df_future['is_weekend'] = df_future['day_of_week'].apply(lambda x: 1 if x >=5 else 0)
    df_future['is_holiday'] = df_future['ds'].dt.date.apply(is_holiday)
    return df_future

# Add external regressors
future_throughput = add_external_regressors(future_throughput)
future_occupancy = add_external_regressors(future_occupancy)

In [ ]:
if FIT_PROPHET:
    forecast_throughput = model_throughput.predict(future_throughput)
    display(forecast_throughput[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head())

    forecast_occupancy = model_occupancy.predict(future_occupancy)
    display(forecast_occupancy[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head())

In [ ]:
if FIT_PROPHET:
    test_throughput = test_throughput.rename(columns={'throughput': 'y'})
    test_throughput = add_external_regressors(test_throughput)

    test_occupancy = test_occupancy.rename(columns={'occupancy': 'y'})
    test_occupancy = add_external_regressors(test_occupancy)

    forecast_test_throughput = model_throughput.predict(test_throughput)
    forecast_test_occupancy = model_occupancy.predict(test_occupancy)

    # Throughput Evaluation
    mae_throughput = mean_absolute_error(test_throughput['y'], forecast_test_throughput['yhat'])
    rmse_throughput = np.sqrt(mean_squared_error(test_throughput['y'], forecast_test_throughput['yhat']))

    print(f'Throughput - MAE: {mae_throughput:.2f}, RMSE: {rmse_throughput:.2f}')

    # Occupancy Evaluation
    mae_occupancy = mean_absolute_error(test_occupancy['y'], forecast_test_occupancy['yhat'])
    rmse_occupancy = np.sqrt(mean_squared_error(test_occupancy['y'], forecast_test_occupancy['yhat']))

    print(f'Occupancy - MAE: {mae_occupancy:.2f}, RMSE: {rmse_occupancy:.2f}')

In [ ]:
if FIT_PROPHET:
    # Throughput Plot
    plt.figure(figsize=(15,5))
    plt.plot(test_throughput['ds'], test_throughput['y'], label='Actual Throughput')
    plt.plot(forecast_test_throughput['ds'], forecast_test_throughput['yhat'], label='Predicted Throughput')
    plt.fill_between(forecast_test_throughput['ds'], 
                    forecast_test_throughput['yhat_lower'], 
                    forecast_test_throughput['yhat_upper'], 
                    color='gray', alpha=0.2, label='Confidence Interval')
    plt.legend()
    plt.xlabel('Time')
    plt.ylabel('Throughput')
    plt.title('Actual vs Predicted Throughput')
    plt.show()

    # Occupancy Plot
    plt.figure(figsize=(15,5))
    plt.plot(test_occupancy['ds'], test_occupancy['y'], label='Actual Occupancy')
    plt.plot(forecast_test_occupancy['ds'], forecast_test_occupancy['yhat'], label='Predicted Occupancy')
    plt.fill_between(forecast_test_occupancy['ds'], 
                    forecast_test_occupancy['yhat_lower'], 
                    forecast_test_occupancy['yhat_upper'], 
                    color='gray', alpha=0.2, label='Confidence Interval')
    plt.legend()
    plt.xlabel('Time')
    plt.ylabel('Occupancy')
    plt.title('Actual vs Predicted Occupancy')
    plt.show()